In [1]:
# preprocessing
from tensorflow.keras import datasets
from tensorflow.keras.backend import expand_dims
from tensorflow.keras.utils import to_categorical

# CNN
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.callbacks import EarlyStopping

2024-06-06 11:42:12.480576: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-06 11:42:12.643001: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-06-06 11:42:12.643022: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-06-06 11:42:12.680271: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-06 11:42:13.845949: W tensorflow/stream_executor/platform/de

# Load data

In [2]:
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data(path="mnist.npz")

(X_train.shape, y_train.shape), (X_test.shape, y_test.shape)

(((60000, 28, 28), (60000,)), ((10000, 28, 28), (10000,)))

# Preprocessing

## Normalize

In [3]:
# scale all pixel intensities to 0-1
X_train = X_train/255
X_test = X_test/255

In [4]:
(X_train.min(), X_train.max()), (X_test.min(), X_test.max())

((0.0, 1.0), (0.0, 1.0))

## Expand dimensions

In [5]:
# reshape tensors to add a single channel for black and white pixels
X_train = expand_dims(X_train, axis = -1)
X_test = expand_dims(X_test, axis = -1)

2024-06-06 11:42:17.014916: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-06-06 11:42:17.015041: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2024-06-06 11:42:17.015114: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Jasmine): /proc/driver/nvidia/version does not exist
2024-06-06 11:42:17.015879: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-06 11:42:17.026668: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 376320000 exceeds 1

In [6]:
X_train.shape, X_test.shape

(TensorShape([60000, 28, 28, 1]), TensorShape([10000, 28, 28, 1]))

## Encode target

In [7]:
# one-hot-encode the number labels
y_train_cat = to_categorical(y_train, num_classes=10)
y_test_cat = to_categorical(y_test, num_classes=10)

In [8]:
y_train_cat.shape, y_test_cat.shape

((60000, 10), (10000, 10))

# CNN 

In [9]:
def initalize_model():
    
    model = models.Sequential()
    
    # convolutional and maxpooling layers
    model.add(layers.Conv2D(8, (4,4), input_shape=(28,28,1), activation="relu", padding="same"))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    
    # flattening layer
    model.add(layers.Flatten())
    
    # classification layer
    model.add(layers.Dense(10, activation="softmax"))
    
    # model compilation
    model.compile(loss="categorical_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
    
    return model

In [10]:
model = initalize_model()

es = EarlyStopping(patience=5)

history = model.fit(X_train, 
                   y_train_cat, 
                   validation_split=0.3,
                   batch_size=32,
                   epochs=100,
                   callbacks= es, 
                   verbose=1)

Epoch 1/100
1313/1313 [==============================] - 9s 7ms/step - loss: 0.3161 - accuracy: 0.9114 - val_loss: 0.1889 - val_accuracy: 0.9448
Epoch 2/100
1313/1313 [==============================] - 9s 7ms/step - loss: 0.1346 - accuracy: 0.9612 - val_loss: 0.1239 - val_accuracy: 0.9642
Epoch 3/100
1313/1313 [==============================] - 12s 9ms/step - loss: 0.1009 - accuracy: 0.9699 - val_loss: 0.1029 - val_accuracy: 0.9716
Epoch 4/100
1313/1313 [==============================] - 13s 10ms/step - loss: 0.0841 - accuracy: 0.9755 - val_loss: 0.0963 - val_accuracy: 0.9710
Epoch 5/100
1313/1313 [==============================] - 11s 8ms/step - loss: 0.0717 - accuracy: 0.9787 - val_loss: 0.0894 - val_accuracy: 0.9729
Epoch 6/100
1313/1313 [==============================] - 13s 10ms/step - loss: 0.0636 - accuracy: 0.9808 - val_loss: 0.0822 - val_accuracy: 0.9751
Epoch 7/100
1313/1313 [==============================] - 13s 10ms/step - loss: 0.0559 - accuracy: 0.9835 - val_loss: 0.0790 

## Evaluate the performance

In [11]:
res = model.evaluate(X_test, y_test_cat, verbose=1)

313/313 [==============================] - 1s 5ms/step - loss: 0.0699 - accuracy: 0.9797
